In [31]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [32]:
if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("deepseek-r1-distill-qwen-32b", model_provider="groq")

In [33]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [34]:
from langchain_chroma import Chroma

vector_store = Chroma(embedding_function=embeddings)

In [41]:
from langchain_community.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import hub
from typing_extensions import TypedDict, List
from langchain.schema import Document
from langgraph.graph import START, StateGraph

loader = JSONLoader(file_path="resume.json", jq_schema='.', text_content=False)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [43]:
response = graph.invoke({"question": "What is your last job?"})
print(response["answer"])

<think>
Okay, I need to figure out what the user's last job was based on the provided context. Let me look through the context carefully.

The context mentions several job titles and companies. The first one is "Graduate Teaching Assistant" at the University of Galway from Sep 2024 - Present. Then there's "Technical Product Manager" at Meituan from Sep 2022 - Sep 2023. After that, there's "Senior Product Manager" at Alibaba Group from Sep 2018 - Sep 2022.

Wait, but the order might be a bit confusing. The user might have listed them in reverse chronological order. Let me check the dates:

- University of Galway: Sep 2024 - Present (current)
- Meituan: Sep 2022 - Sep 2023
- Alibaba: Sep 2018 - Sep 2022

So, the most recent job is "Graduate Teaching Assistant" at the University of Galway. But the user is asking for the "last job," which could mean the most recent one before the current position. Alternatively, if "Graduate Teaching Assistant" is the current role, that would be the last j